# Chromosome 1 analysis

## Loading the data

Import the necessary packages:

In [1]:
import pandas as pd
import numpy as np
from trails.optimizer import trans_emiss_calc, post_prob_wrapper
from trails.read_data import maf_parser
from trails.cutpoints import cutpoints_ABC
%load_ext rpy2.ipython

n_int_AB_original = 3
n_int_ABC_original = 3

n_int_AB = 3
n_int_ABC = 3

Load the data:

In [2]:
# Read the output from TRAILS
df = pd.read_csv('../results/chr1_L-BFGS-B.csv')
# Find iteration with largest likelihood
df = df[df['loglik'] == df['loglik'].max()]
# Convert parameter estimates into dictionary
dct = dict(zip(list(df.columns), df.iloc[0].to_list()))

In [3]:
# Print dictionary
dct

{'n_eval': 252.0,
 't_A': 0.0029307448136187,
 't_B': 0.0030864038918802,
 't_C': 0.005659606389834,
 't_2': 0.0020879423249091,
 't_upper': 0.0049568487930341,
 'N_AB': 0.0034232595068766,
 'N_ABC': 0.002807779932867,
 'r': 0.9920256455623097,
 'loglik': -76112721.76133902,
 'time': 153659.25739312172}

The following lines can be used in case we would like to change from certain `n_int_AB` and `n_int_ABC` to another:

In [4]:
cut_ABC_original = (cutpoints_ABC(n_int_ABC_original, 1)*dct["N_ABC"])

In [5]:
t_ABC = cut_ABC_original[-2]+dct["t_upper"]
t_ABC

0.008041510331157506

In [6]:
cut_ABC = (cutpoints_ABC(n_int_ABC, 1)*dct["N_ABC"])
cut_ABC_last = cut_ABC[-2]
t_upper = t_ABC-cut_ABC_last
t_upper

0.0049568487930341

In [7]:
t_out = (((dct["t_A"]+dct["t_B"])/2+dct["t_2"])+dct["t_C"])/2 + cut_ABC_original[n_int_ABC_original-1]*dct["N_ABC"] + dct["t_upper"] + 2*dct["N_ABC"]

In [8]:
t_out

0.015959131243280803

## Parameter estimation

First, we can calculate the proportion of ILS from the estimated parameters. We can use the following function:

$\frac{2}{3}\exp\left(-\frac{t_2}{N_\text{AB}}\right)$

Since the units of $t_2$ are number of generations (multiplied by the mutation rate) and the units of $N_\text{AB}$ are number of individuals times two (multiplied by the mutation rate), then we can directly use the formula above:

In [9]:
(2/3)*np.exp(-(dct["t_2"])/(dct["N_AB"]))

0.36225994154321695

We can convert the raw units of the times into number of generations by dividing it by the mutation rate, and into years by then multiplying it by the generation time:

In [15]:
df_params = pd.DataFrame()
df_params['Names'] = ['H to HC', 'C to HC', 'HC to HCG', 'G to HCG', 'O (divergence)']
df_params['Raw values'] = [dct[i] for i in ['t_A', 't_B', 't_2', 't_C']] + [t_out]
df_params['Generations (mu = 1.25e-8)'] = df_params['Raw values']/1.25e-8
df_params['MY (mu = 1.25e-8, g = 25)'] = df_params['Raw values']/1.25e-8*25/1000000
df_params

,Names,Raw values,Generations (mu = 1.25e-8),"MY (mu = 1.25e-8, g = 25)"
0,H to HC,0.002931,2.344596e+05,5.861490
1,C to HC,0.003086,2.469123e+05,6.172808
2,HC to HCG,0.002088,1.670354e+05,4.175885
3,G to HCG,0.005660,4.527685e+05,11.319213
4,O (divergence),0.015959,1.276730e+06,31.918262


We can also convert the effective population sizes into diploid population size by dividing by 2 times the mutation rate:

In [13]:
df_params = pd.DataFrame()
df_params['Names'] = ['HC', 'HCG']
df_params['Raw values'] = [dct[i] for i in ['N_AB', 'N_ABC']]
df_params['N (mu = 1e-8)'] = df_params['Raw values']/1.25e-8/2
df_params

,Names,Raw values,N (mu = 1e-8)
0,HC,0.003423,136930.380275
1,HCG,0.002808,112311.197315


Finally, the recombination rate can be calculated by multiplying the raw value by the mutation rate:

In [17]:
df_params = pd.DataFrame()
df_params['Names'] = ['Recombination']
df_params['Raw value'] = [dct[i] for i in ['r']]
df_params['r (mu = 1.25e-8)'] = df_params['Raw value']*1.25e-8
df_params

,Names,Raw value,r (mu = 1.25e-8)
0,Recombination,0.992026,1.240032e-08


## Posterior decoding

Based on the estimated values, we can compute the transition and emission probabilities of the TRAILS model:

In [19]:
a, b, pi, hidden_names, observed_names = trans_emiss_calc(
    dct["t_A"], dct["t_B"], dct["t_C"], dct["t_2"], t_upper, t_out, 
    dct["N_AB"], dct["N_ABC"], dct["r"], n_int_AB, n_int_ABC)

2023-04-25 10:40:10,160	INFO worker.py:1518 -- Started a local Ray instance.


We can then parse the chr1 region using the TRAILS maf parser:

In [21]:
if 'obs' not in globals():
    obs = maf_parser('../data/chr1.filtered.region.maf', ['hg38','panTro5','gorGor5','ponAbe2'])

To calculate the posterior probabilities, we can use the `post_prob_wrapper` function:

In [22]:
post_prob = post_prob_wrapper(a, b, pi, obs)

In [23]:
len(post_prob)

2599

The following function is used to extract the sequence and position information from the maf alignment:

In [25]:
from Bio import AlignIO

def maf_seq(file, sp_lst):
    total_dct = {sp_lst[0]:[], sp_lst[1]:[], sp_lst[2]:[], sp_lst[3]:[]}
    start_dct = []
    # Start loglik accumulator
    loglik_acc = 0
    # For each block
    for multiple_alignment in AlignIO.parse(file, "maf"):
        # Save sequence
        dct = {}
        # For each sequence in the alignment block
        for seqrec in multiple_alignment:
            if seqrec.name.split('.')[0] in sp_lst:
                # Save nucleotides as ones and gaps as zeros
                dct[seqrec.name.split('.')[0]] = ''.join([str(int(j!='-')) for j in str(seqrec.seq)])
            if seqrec.name.split('.')[0] == 'hg38':
                # Save the start position of the block in human coordinates
                start = seqrec.annotations['start']
        # Append information to list
        if len(dct) == 4: 
            start_dct.append(start)
            for i in sp_lst:
                total_dct[i].append(dct[i])
    return total_dct, start_dct

# Get sequence and start information
seqinfo, startinfo = maf_seq('../data/chr1.filtered.region.maf', ['hg38','panTro5','gorGor5','ponAbe2'])

Now, we will iterate over the alignment blocks, save the position in human coordinates, and mark gaps as -9:

In [26]:
tot_lst = []
# For each block
for i in range(len(startinfo)):
    # Save start coordinate
    st = startinfo[i]
    tmp_lst = []
    # For each site in the human sequence
    for j in seqinfo['hg38'][i]:
        # If the site is not a gap
        if j == '1':
            # Append position
            tmp_lst.append(st)
            # Update coordinate
            st += 1
        # If it is a gap
        else:
            # Append -9
            tmp_lst.append(-9)
    tot_lst.append(tmp_lst)

In order to summarize the data, we will calculate the mean posterior probability in 100 kb windows for each of the hidden states:

In [27]:
# Create empty data frame
summary = pd.DataFrame()
# Define bin size
binsize = 100000
# For each block
for j in range(len(post_prob)):
    # Convert posteriors into data frame
    dat = pd.DataFrame(post_prob[j])
    # Add hidden states as column names
    dat.columns = [str(i) for i in list(hidden_names.values())]
    # Add position in human coordinates
    dat['position'] = tot_lst[j]
    # Filter gaps
    dat = dat[dat['position'] != -9]
    # Bin the result using the bin size
    dat['bin'] = dat['position'] // binsize
    # Calculate number of sites per bin
    t1 = dat.groupby('bin').agg(size=('(0, 0, 0)', 'size')).reset_index()
    # Calculate mean posterior probability per hidden state
    t2 = dat.groupby('bin').agg('mean').reset_index()
    # Append to overall result
    summary = pd.concat([summary, t1.merge(t2)])

In [21]:
binsize = 100000

We can import the data frame into R for plotting:

In [19]:
%%R 

library(tidyverse)

# summary %>% write_csv('../data/summary.csv')

summary <- read_csv('../data/summary.csv')

totab <- summary %>%
    # Convert to tibble
    as_tibble() %>%
    # Remove unused columns
    select(-position) %>%
    # Pivot table
    pivot_longer(-c(bin, size)) %>%
    # For each bin and hidden state
    group_by(bin, name) %>%
    summarize(
        # Calculate the mean posterior weighted by 
        # the number of sites
        value = weighted.mean(value, size),
        size = sum(size)
    ) %>%
    # Convert hidden state names into plottable info
    mutate(
        name = str_remove_all(name, '\\('),
        name = str_remove_all(name, '\\)')
    ) %>%
    separate(name, c('V', 't1', 't2'), ', ') %>%
    mutate(
        V = as.integer(V),
        t1 = as.integer(t1),
        t2 = as.integer(t2),
        is_V0 = V == 0
    ) 
totab 

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Rows: 3053 Columns: 30
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (30): bin, size, (0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 0), (0, 1, 1), ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
`summarise()` has grouped output by 'bin'. You can override using the `.groups` argument.
# A tibble: 13,554 × 7
# Groups:   bin [502]
     bin     V    t1    t2  value  size is_V0
   <dbl> <int> <int> <int>  <dbl> <dbl> <lgl>
 1   249     0     0     0 0.0594  1485 TRUE 
 2   249     0     0     1 0.0454  1485 TRUE 
 3   249     0     0     2 0.0325  1485 TRUE 
 4   249     0     1     0 0.0633  1485 TRUE 
 5   249     0     1     1 0.0483  1485 TRUE 
 6   249     0     1     2 0.0349  1485 TRUE 
 7   249     0     2     0 0.0655  1485 TRUE 
 8   249     0     2     1 0.0499  1485 TRUE 
 9   249     0     2     2 0.0367  1485 TRUE 
10   

In [30]:
%%R -w 2000 -h 150 -r 150 -i binsize

# Load packages
library(ggHoriPlot)
library(ggthemes)

p_ILS <- totab %>%
    # Group by bin and topology
    group_by(bin, V) %>%
    # Sum the mean posterior
    summarize(
        value = sum(value)
    ) %>%
    # Group only by bin
    group_by(bin) %>%
    # Calculate ILS proxy from the mean posterior
    summarize(
        ILS = sum(value[V %in% c(2, 3)])/sum(value)
    ) %>%
    # Convert bins into coordinates
    mutate(bin = bin*binsize/1000000) %>%
    # Plot as horizon plot
    ggplot() +
    geom_horizon(aes(x = bin, xend = bin+(binsize-1)/1000000, y = ILS,
                     fill=..Cutpoints..), horizonscale = 4, rm.outliers = T) +
    theme_few() +
    scale_fill_hcl(palette = 'PuOr', labels = c('High', '', '', 'Low'), name = 'ILS') +
    theme(axis.title.x=element_blank(),
        axis.text.x=element_blank(),
        axis.ticks.x=element_blank(),
        axis.title.y=element_blank(),
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank()) +
    scale_y_continuous(expand = c(0, 0)) +
    scale_x_continuous(expand = c(0, 0))

`summarise()` has grouped output by 'bin'. You can override using the `.groups` argument.


In [31]:
%%R -w 2000 -h 300 -r 150

# This is similar to the previous block, 
# but for the second coalescent
p_t2 <- totab %>%
    group_by(bin, t2) %>%
    summarize(
        value = sum(value)
    ) %>%
    group_by(bin) %>%
    summarize(value = weighted.mean(t2, value)) %>%
    mutate(bin = bin*binsize/1000000) %>%
    ggplot() +
    geom_horizon(aes(x = bin, xend = bin+(binsize-1)/1000000, y = value, fill=..Cutpoints..), 
                 horizonscale = 4, rm.outliers = T) +
    theme_few() +
    scale_fill_hcl(
        palette = 'PuOr', 
        labels = c('Deep', '', '', 'Young'), 
        name = 'Second\ncoalescent'
    ) +
    theme(axis.title.x=element_blank(),
        axis.text.x=element_blank(),
        axis.ticks.x=element_blank(),
        axis.title.y=element_blank(),
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank()) +
    scale_y_continuous(expand = c(0, 0)) +
    scale_x_continuous(expand = c(0, 0))

`summarise()` has grouped output by 'bin'. You can override using the `.groups` argument.


In [32]:
%%R -w 2000 -h 300 -r 150 -i n_int_AB

# This is similar to the previous block, 
# but for the first coalescent
p_t1 <- totab %>%
    group_by(bin, t1, is_V0) %>%
    summarize(
        value = sum(value)
    ) %>%
    ungroup() %>%
    mutate(
        t1 = ifelse(!is_V0, t1+n_int_AB, t1)
    ) %>%
    group_by(bin) %>%
    summarize(value = weighted.mean(t1, value)) %>%
    mutate(bin = bin*binsize/1000000) %>%
    ggplot() +
    geom_horizon(aes(x = bin, xend = bin+(binsize-1)/1000000, y = value, fill=..Cutpoints..), 
                 horizonscale = 4, rm.outliers = T) +
    theme_few() +
    scale_fill_hcl(palette = 'PuOr', labels = c('Deep', '', '', 'Young'), name = 'First\ncoalescent') +
    theme(
        axis.title.y=element_blank(),
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank()
    ) +
    scale_y_continuous(expand = c(0, 0)) +
    scale_x_continuous(expand = c(0, 0)) +
    xlab('Position along human chr1 (Mb)')

`summarise()` has grouped output by 'bin', 't1'. You can override using the `.groups` argument.


In [47]:
%%R -w 2000 -h 800 -r 150

library(patchwork)

p_ILS / p_t2 / p_t1 + plot_layout(guides = 'collect')

ggsave('../plots/horizon_chr1.pdf', height = 5, width = 16)

In [129]:
%%R -w 2000 -h 150 -r 150 -i binsize

# Load packages
library(ggHoriPlot)
library(ggthemes)

tab_ILS <- totab %>%
    # Group by bin and topology
    group_by(bin, V) %>%
    # Sum the mean posterior
    summarize(
        value = sum(value)
    ) %>%
    # Group only by bin
    group_by(bin) %>%
    # Calculate ILS proxy from the mean posterior
    summarize(
        ILS = sum(value[V %in% c(2, 3)])/sum(value)
    ) %>%
    # Convert bins into coordinates
    mutate(bin = bin*binsize/1000000) %>%
    ungroup() %>%
    mutate(
        ILS = case_when(
            ILS <= quantile(ILS, 0.01) ~ min(ILS[ILS > quantile(ILS, 0.01)]),
            ILS >= quantile(ILS, 0.99) ~ max(ILS[ILS < quantile(ILS, 0.99)]),
            TRUE ~ ILS
        )
    )
    
p_ILS <- tab_ILS %>%
    # Plot as horizon plot
    ggplot() +
    geom_tile(aes(x = bin, y = 0, fill=ILS)) +
    theme_few() +
    theme(axis.title.x=element_blank(),
        axis.text.x=element_blank(),
        axis.ticks.x=element_blank(),
        axis.title.y=element_blank(),
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        legend.direction="horizontal",
        #legend.text = element_blank()
         ) +
    scale_y_continuous(expand = c(0, 0)) +
    scale_x_continuous(expand = c(0, 0)) +
    colorspace::scale_fill_continuous_diverging(
        name = 'ILS',
        palette = "Blue-Red 3", mid = (max(tab_ILS$ILS)+min(tab_ILS$ILS))/2,
        breaks = c(min(tab_ILS$ILS)+(max(tab_ILS$ILS)-min(tab_ILS$ILS))*0.1, 
                   min(tab_ILS$ILS)+(max(tab_ILS$ILS)-min(tab_ILS$ILS))*0.9),
        labels = c('Low', 'High')) + 
    guides(fill=guide_colorbar(ticks.colour = NA, title.position="top", title.hjust = 0.5))

`summarise()` has grouped output by 'bin'. You can override using the `.groups` argument.


In [140]:
%%R -w 2000 -h 300 -r 150

# This is similar to the previous block, 
# but for the second coalescent
tab_t2 <- totab %>%
    group_by(bin, t2) %>%
    summarize(
        value = sum(value)
    ) %>%
    group_by(bin) %>%
    summarize(t2 = weighted.mean(t2, value)) %>%
    mutate(bin = bin*binsize/1000000) %>%
    ungroup() %>%
    mutate(
        t2 = case_when(
            t2 <= quantile(t2, 0.01) ~ min(t2[t2 > quantile(t2, 0.01)]),
            t2 >= quantile(t2, 0.99) ~ max(t2[t2 < quantile(t2, 0.99)]),
            TRUE ~ t2
        )
    )
    
p_t2 <- tab_t2 %>%
    ggplot() +
    geom_tile(aes(x = bin, y = 0, fill=t2)) +
    theme_few() +
    theme(
        axis.title.y=element_blank(),
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        legend.direction="horizontal",
        # legend.text = element_blank()
    )  +
    scale_y_continuous(expand = c(0, 0)) +
    scale_x_continuous(expand = c(0, 0)) +
    colorspace::scale_fill_continuous_diverging(
        name = '2nd coalescent',
        palette = "Blue-Red 3", mid = (max(tab_t2$t2)+min(tab_t2$t2))/2,
        breaks = c(min(tab_t2$t2)+(max(tab_t2$t2)-min(tab_t2$t2))*0.1, 
                   min(tab_t2$t2)+(max(tab_t2$t2)-min(tab_t2$t2))*0.9),
        labels = c('Young', 'Deep')) + 
    guides(fill=guide_colorbar(ticks.colour = NA, title.position="top", title.hjust = 0.5))

`summarise()` has grouped output by 'bin'. You can override using the `.groups` argument.


In [144]:
%%R -w 2000 -h 300 -r 150 -i n_int_AB

# This is similar to the previous block, 
# but for the first coalescent
tab_t1 <- totab %>%
    group_by(bin, t1, is_V0) %>%
    summarize(
        value = sum(value)
    ) %>%
    ungroup() %>%
    mutate(
        t1 = ifelse(!is_V0, t1+n_int_AB, t1)
    ) %>%
    group_by(bin) %>%
    summarize(t1 = weighted.mean(t1, value)) %>%
    mutate(bin = bin*binsize/1000000) %>%
    mutate(
        t1 = case_when(
            t1 <= quantile(t1, 0.01) ~ min(t1[t1 > quantile(t1, 0.01)]),
            t1 >= quantile(t1, 0.99) ~ max(t1[t1 < quantile(t1, 0.99)]),
            TRUE ~ t1
        )
    )
    
p_t1 <- tab_t1 %>%
    ggplot() +
    geom_tile(aes(x = bin, y = 0, fill=t1)) +
    theme_few() +
    theme(axis.title.x=element_blank(),
        axis.text.x=element_blank(),
        axis.ticks.x=element_blank(),
        axis.title.y=element_blank(),
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        legend.direction="horizontal",
        # legend.text = element_blank()
         ) +
    scale_y_continuous(expand = c(0, 0)) +
    scale_x_continuous(expand = c(0, 0)) +
    xlab('Position along human chr1 (Mb)') +
    colorspace::scale_fill_continuous_diverging(
        name = '1st coalescent',
        palette = "Blue-Red 3", mid = (max(tab_t1$t1)+min(tab_t1$t1))/2,
        breaks = c(min(tab_t1$t1)+(max(tab_t1$t1)-min(tab_t1$t1))*0.1, 
                   min(tab_t1$t1)+(max(tab_t1$t1)-min(tab_t1$t1))*0.9),
        labels = c('Young', 'Deep')
    ) + 
    guides(fill=guide_colorbar(ticks.colour = NA, title.position="top", title.hjust = 0.5))

`summarise()` has grouped output by 'bin', 't1'. You can override using the `.groups` argument.


In [145]:
%%R -w 2000 -h 600 -r 150

library(patchwork)

p_ILS / p_t1 / p_t2 & theme(plot.margin = unit(c(0,0,0,0), "cm"))

ggsave('../plots/heatmap_chr1.pdf', height = 3, width = 16)

In [159]:
%%R

full_join(full_join(tab_ILS, tab_t1, by = 'bin'), tab_t2, by = 'bin') %>%
    summarize(
        ILS_t1 = cor(ILS, t1),
        ILS_t2 = cor(ILS, t2),
        t1_t2 = cor(t1, t2)
    ) 

# A tibble: 1 × 3
  ILS_t1 ILS_t2 t1_t2
   <dbl>  <dbl> <dbl>
1  0.982  0.517 0.536
